## Llenando tabla Equipos

In [410]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    dbname="nba_db", # <- Aqui va el nombre de tu base de datos
    user="postgres", # <- Aqui va tu usuario de pgadmin
    password="12345" # <- Aqui va la contraseña
)
cursor = conn.cursor()

In [411]:
nba_conference = {
    "ATL": "Este",   # Atlanta Hawks
    "BOS": "Este",   # Boston Celtics
    "BKN": "Este",   # Brooklyn Nets
    "CHA": "Este",   # Charlotte Hornets
    "CHI": "Este",   # Chicago Bulls
    "CLE": "Este",   # Cleveland Cavaliers
    "DET": "Este",   # Detroit Pistons
    "IND": "Este",   # Indiana Pacers
    "MIA": "Este",   # Miami Heat
    "MIL": "Este",   # Milwaukee Bucks
    "NYK": "Este",   # New York Knicks
    "ORL": "Este",   # Orlando Magic
    "PHI": "Este",   # Philadelphia 76ers
    "TOR": "Este",   # Toronto Raptors
    "WAS": "Este",   # Washington Wizards

    "DAL": "Oeste",  # Dallas Mavericks
    "DEN": "Oeste",  # Denver Nuggets
    "GSW": "Oeste",  # Golden State Warriors
    "HOU": "Oeste",  # Houston Rockets
    "LAC": "Oeste",  # LA Clippers
    "LAL": "Oeste",  # Los Angeles Lakers
    "MEM": "Oeste",  # Memphis Grizzlies
    "MIN": "Oeste",  # Minnesota Timberwolves
    "NOP": "Oeste",  # New Orleans Pelicans
    "OKC": "Oeste",  # Oklahoma City Thunder
    "PHX": "Oeste",  # Phoenix Suns
    "POR": "Oeste",  # Portland Trail Blazers
    "SAC": "Oeste",  # Sacramento Kings
    "SAS": "Oeste",  # San Antonio Spurs
    "UTA": "Oeste"   # Utah Jazz
}

In [412]:
from nba_api.stats.static import teams

# Obtener equipos
nba_teams = teams.get_teams()

for team in nba_teams:
    id = team['id']
    nombre = team['full_name']
    abbrev = team['abbreviation']
    ciudad = team['city']
    estado = team['state']
    conferencia = nba_conference.get(abbrev)
    anio_fundado = team['year_founded']

    cursor.execute("""
        INSERT INTO equipo (id_equipo, NombreEquipo, Ciudad, Estado, Abbrev, Conferencia, AnioFundado)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (NombreEquipo) DO NOTHING
    """, (id, nombre, ciudad, estado, abbrev, conferencia, anio_fundado))

conn.commit()

## Llenando Tabla Temporada

In [413]:
# Conexión a la base de datos
conn = psycopg2.connect(
    host="localhost",
    dbname="nba_db",
    user="postgres",
    password="12345"
)
cursor = conn.cursor()

In [414]:
try:
    cursor.execute("""
        INSERT INTO temporada (NombreTemporada, FechaInicio, FechaFinal, Campeon)
        VALUES 
        (%s, %s, %s, %s),
        (%s, %s, %s, %s)
        ON CONFLICT (NombreTemporada) DO NOTHING
    """, (
        '2021-22', '2021-10-19', '2022-06-16', 1610612744,
        '2022-23', '2022-10-18', '2023-06-12', 1610612743
    ))
    conn.commit()
    print("Inserción exitosa")
except Exception as e:
    conn.rollback()
    print(f"Error al insertar: {e}")

Inserción exitosa


## Llenado tabla Partidos

In [415]:
abbrev_a_id = {
    "ATL": 1610612737,
    "BOS": 1610612738,
    "CLE": 1610612739,
    "NOP": 1610612740,
    "CHI": 1610612741,
    "DAL": 1610612742,
    "DEN": 1610612743,
    "GSW": 1610612744,
    "HOU": 1610612745,
    "LAC": 1610612746,
    "LAL": 1610612747,
    "MIA": 1610612748,
    "MIL": 1610612749,
    "MIN": 1610612750,
    "BKN": 1610612751,
    "NYK": 1610612752,
    "ORL": 1610612753,
    "IND": 1610612754,
    "PHI": 1610612755,
    "PHX": 1610612756,
    "POR": 1610612757,
    "SAC": 1610612758,
    "SAS": 1610612759,
    "OKC": 1610612760,
    "TOR": 1610612761,
    "UTA": 1610612762,
    "MEM": 1610612763,
    "WAS": 1610612764,
    "DET": 1610612765,
    "CHA": 1610612766,
}

In [416]:
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd
import numpy as np

def get_and_clean_df(temporada_id=22023, abbrev_a_id=None):
    gamefinder = leaguegamefinder.LeagueGameFinder(
        season_type_nullable='Regular Season',league_id_nullable='00'
    )

    games_df = gamefinder.get_data_frames()[0]

    # Filtrar por temporada (hacemos una copia explícita para evitar el warning)
    games_temporada = games_df[games_df['SEASON_ID'] == temporada_id].copy()

    # Columna para guardar la abbrev del equipo visitante
    games_temporada.loc[:, 'Visitante'] = games_temporada['MATCHUP'].apply(lambda x: x[-3:])

    # Columna donde se guarda el id del visitante buscado por su abbrev
    games_temporada.loc[:, 'Visitante_ID'] = games_temporada['Visitante'].map(abbrev_a_id).astype(str)

    # Se guarda el nombre de la temporada basado en su ID 
    games_temporada['SEASON_NAME'] = games_temporada['SEASON_ID'].map({
    '22023': '2023-24',
    '22022': '2022-23',
    '22021': '2021-22',
    '22020': '2020-21',
    '22019': '2019-20'
})
    
    # Columna donde guardamos el ID del ganador
    games_temporada.loc[:, 'ganador'] = np.where(
        games_temporada['WL'] == 'W',
        games_temporada['TEAM_ID'],
        games_temporada['Visitante_ID']
    )


    # Eliminar duplicados
    unique_games = games_temporada.drop_duplicates(subset=['GAME_ID'])

    return unique_games[['SEASON_NAME','GAME_ID', 'TEAM_ID', 'Visitante_ID','GAME_DATE','ganador']][:50]


In [417]:
get_and_clean_df('22023', abbrev_a_id).shape

(50, 6)

In [419]:
temporadas = ('22021', '22022')

for temporada in temporadas:
    temporada_df = get_and_clean_df(temporada, abbrev_a_id)

    for _, row in temporada_df.iterrows():
        id_partido = row['GAME_ID']
        EquipoCasa = row['TEAM_ID']
        EquipoVisitante = row['Visitante_ID']
        Fecha = row['GAME_DATE']
        ganador = row['ganador']
        NombreTemporada = row['SEASON_NAME']

        try:
            cursor.execute("""
                INSERT INTO partido (id_partido, EquipoCasa, EquipoVisitante, NombreTemporada, FechaPartido, Ganador)
                VALUES (%s, %s, %s, %s, %s, %s)
                ON CONFLICT (id_partido) DO NOTHING
            """, (id_partido, EquipoCasa, EquipoVisitante, NombreTemporada, Fecha, ganador))
        except Exception as e:
            print(f"Error inserting game {id_partido}: {e}")
            conn.rollback()
        else:
            conn.commit()

## Llenando tabla Jugador

In [420]:
from nba_api.stats.endpoints import commonallplayers

def get_players(season='2021-22'):
    all_players = commonallplayers.CommonAllPlayers(
        is_only_current_season=1,
        season=season
    )
    df_players = all_players.get_data_frames()[0]
    df_players['NOMBRE'] = df_players['DISPLAY_FIRST_LAST'].apply(lambda x: x.split(' ')[0])
    df_players['APELLIDO'] = df_players['DISPLAY_FIRST_LAST'].apply(lambda x: x.split(' ')[1])
    df_players = df_players[df_players['TEAM_ID'] != 0]
    df_players['SEASON_ID'] = season
    df_players = df_players[['PERSON_ID', 'NOMBRE', 'APELLIDO','TEAM_ID', 'SEASON_ID','TEAM_ABBREVIATION']][:50]
    return df_players


In [421]:
jugadores_21_22 = get_players(season='2021-22')
jugadores_22_23 = get_players(season='2022-23')

In [422]:
import time
from nba_api.stats.endpoints import commonplayerinfo

def agregar_altura_al_df(players_df):
    alturas_cm = []

    for _, row in players_df.iterrows():
        player_id = row['PERSON_ID']
        nombre = row['NOMBRE']
        apellido = row['APELLIDO']

        try:
            # Obtener la información detallada del jugador
            info = commonplayerinfo.CommonPlayerInfo(player_id=player_id).get_data_frames()[0]
            altura_str = info['HEIGHT'].values[0]  # Ej: '6-7'

            # Convertir a centímetros
            if isinstance(altura_str, str) and '-' in altura_str:
                pies, pulgadas = map(int, altura_str.split('-'))
                altura_cm = round((pies * 12 + pulgadas) * 2.54)
            else:
                altura_cm = None

            alturas_cm.append(altura_cm)
            print(f"✔ {nombre} {apellido} → {altura_cm} cm")
            time.sleep(1.2)

        except Exception as e:
            print(f"⚠ Error con {nombre} {apellido}: {e}")
            alturas_cm.append(None)
            time.sleep(3)

    # Agregar la columna al DataFrame
    players_df['ALTURA_CM'] = alturas_cm
    return (players_df, alturas_cm)

In [423]:
''''
df_players_final_22, alturas = agregar_altura_al_df(jugadores_22_23)
df_players_final_22.to_csv('jugadores_22_23.csv', index=False)

df_players_final_21, alturas = agregar_altura_al_df(jugadores_21_22)
df_players_final_21.to_csv('jugadores_21_22.csv', index=False)
'''


"'\ndf_players_final_22, alturas = agregar_altura_al_df(jugadores_22_23)\ndf_players_final_22.to_csv('jugadores_22_23.csv', index=False)\n\ndf_players_final_21, alturas = agregar_altura_al_df(jugadores_21_22)\ndf_players_final_21.to_csv('jugadores_21_22.csv', index=False)\n"

In [424]:
df_players_21 = pd.read_csv('jugadores_21_22.csv')
df_players_22 = pd.read_csv('jugadores_22_23.csv')
df_players_final = pd.concat([df_players_21, df_players_22], ignore_index=True)

In [425]:
df_players_final

,PERSON_ID,NOMBRE,APELLIDO,TEAM_ID,SEASON_ID,TEAM_ABBREVIATION,ALTURA_CM
0,200746,LaMarcus,Aldridge,1610612751,2021-22,BKN,211
1,2546,Carmelo,Anthony,1610612747,2021-22,LAL,201
2,1627853,Ryan,Arcidiacono,1610612752,2021-22,NYK,190
3,201571,D.J.,Augustin,1610612747,2021-22,LAL,180
4,203145,Kent,Bazemore,1610612747,2021-22,LAL,193
...,...,...,...,...,...,...,...
95,1630177,Theo,Maledon,1610612766,2022-23,CHA,193
96,203585,Rodney,McGruder,1610612765,2022-23,DET,193
97,203526,Raul,Neto,1610612739,2022-23,CLE,188
98,1629669,Jaylen,Nowell,1610612750,2022-23,MIN,193


In [427]:

for _, row in df_players_final.iterrows():
    player_id = row['PERSON_ID']
    nombre = row['NOMBRE']
    apellido = row['APELLIDO']
    altura_cm = row['ALTURA_CM']
    try:
        cursor.execute("""
            INSERT INTO jugador (id_jugador, NombreJugador, ApellidoJugador, Altura_cm)
            VALUES (%s, %s, %s, %s)
            ON CONFLICT (id_jugador) DO NOTHING
        """, (player_id, nombre, apellido, altura_cm))
    except Exception as e:
        print(f"Error inserting player {player_id}: {e}")
        conn.rollback()
    else:
        conn.commit()

## Llenando tabla JugadorTemporada

In [428]:
# Make a copy of the original DataFrame
df_reversed = df_players_final.copy()

# Define the mapping to swap the season IDs
swap_dict = {
    '2021-22': '2022-23',
    '2022-23': '2021-22'
}

# Replace the values using the mapping
df_reversed['SEASON_ID'] = df_reversed['SEASON_ID'].map(swap_dict)

In [429]:
df_temporada = pd.concat([df_players_final, df_reversed], ignore_index=True)
df_temporada

,PERSON_ID,NOMBRE,APELLIDO,TEAM_ID,SEASON_ID,TEAM_ABBREVIATION,ALTURA_CM
0,200746,LaMarcus,Aldridge,1610612751,2021-22,BKN,211
1,2546,Carmelo,Anthony,1610612747,2021-22,LAL,201
2,1627853,Ryan,Arcidiacono,1610612752,2021-22,NYK,190
3,201571,D.J.,Augustin,1610612747,2021-22,LAL,180
4,203145,Kent,Bazemore,1610612747,2021-22,LAL,193
...,...,...,...,...,...,...,...
195,1630177,Theo,Maledon,1610612766,2021-22,CHA,193
196,203585,Rodney,McGruder,1610612765,2021-22,DET,193
197,203526,Raul,Neto,1610612739,2021-22,CLE,188
198,1629669,Jaylen,Nowell,1610612750,2021-22,MIN,193


In [430]:
for _, row in df_temporada.iterrows():
    player_id = row['PERSON_ID']
    Equipo = row['TEAM_ID']
    NombreTemporada = row['SEASON_ID']
    try:
        cursor.execute("""
            INSERT INTO jugadorTemporada (id_jugador, NombreTemporada, Equipo)
            VALUES (%s, %s, %s)
            ON CONFLICT (id_jugador, NombreTemporada) DO NOTHING
        """, (player_id, NombreTemporada, Equipo))
    except Exception as e:
        print(f"Error inserting player {player_id}: {e}")
        conn.rollback()
    else:
        conn.commit()

## Llenando EstadisticasJugador

In [431]:
import time
from nba_api.stats.endpoints import playercareerstats

def agregar_estadisticas_temporada(players_df, temporada='2022-23'):
    # Lista donde se almacenarán los diccionarios de estadísticas
    estadisticas = []

    for _, row in players_df.iterrows():
        player_id = row['PERSON_ID']
        nombre = row['NOMBRE']
        apellido = row['APELLIDO']

        try:
            # Obtener estadísticas de carrera por temporada
            stats_df = playercareerstats.PlayerCareerStats(player_id=player_id).get_data_frames()[0]

            # Filtrar por la temporada deseada
            stats_temp = stats_df[stats_df['SEASON_ID'] == temporada]

            if not stats_temp.empty:
                fila = stats_temp.iloc[0]
                estadisticas.append({
                    'PERSON_ID': player_id,
                    'SEASON_ID': temporada,
                    'GP': fila['GP'],
                    'PTS': fila['PTS'],
                    'AST': fila['AST'],
                    'REB': fila['REB'],
                    'BLK': fila['BLK'],
                    'MIN': fila['MIN'],
                    'FG_PCT': fila['FG_PCT'],
                    'FG3_PCT': fila['FG3_PCT'],
                    'FT_PCT': fila['FT_PCT']
                })
                print(f"✔ {nombre} {apellido} – {temporada}")
            else:
                print(f"⚠ {nombre} {apellido} no jugó en {temporada}")
                continue

            time.sleep(1.2)

        except Exception as e:
            print(f"⚠ Error con {nombre} {apellido}: {e}")
            estadisticas.append({
                'PERSON_ID': player_id,
                'SEASON_ID': temporada,
                'GP': None,
                'PTS': None,
                'AST': None,
                'REB': None,
                'BLK': None,
                'MIN': None,
                'FG_PCT': None,
                'FG3_PCT': None,
                'FT_PCT': None
            })
            time.sleep(3)

    # Crear DataFrame de estadísticas y hacer merge con players_df
    stats_df = pd.DataFrame(estadisticas)
    players_stats_df = players_df.merge(stats_df, on='PERSON_ID', how='left')
    
    return players_stats_df

In [432]:
#players_stats_df_23 = agregar_estadisticas_temporada(df_players_final.drop(columns=['SEASON_ID']), '2022-23')
#players_stats_df_23.to_csv('jugadores_estadisticas_23.csv', index=False)

players_stats_df_23 = pd.read_csv('jugadores_estadisticas_23.csv')


In [433]:
players_stats_df_23_clean = players_stats_df_23.drop_duplicates(subset=['PERSON_ID'])

In [434]:
players_stats_df_23_clean

,PERSON_ID,NOMBRE,APELLIDO,TEAM_ID,TEAM_ABBREVIATION,ALTURA_CM,SEASON_ID,GP,PTS,AST,REB,BLK,MIN,FG_PCT,FG3_PCT,FT_PCT
0,200746,LaMarcus,Aldridge,1610612751,BKN,211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2546,Carmelo,Anthony,1610612747,LAL,201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1627853,Ryan,Arcidiacono,1610612752,NYK,190,2022-23,11.0,3.0,2.0,4.0,0.0,26.0,0.200,0.333,0.000
3,201571,D.J.,Augustin,1610612747,LAL,180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,203145,Kent,Bazemore,1610612747,LAL,193,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1630177,Theo,Maledon,1610612766,CHA,193,2022-23,44.0,295.0,152.0,123.0,12.0,854.0,0.402,0.295,0.851
96,203585,Rodney,McGruder,1610612765,DET,193,2022-23,32.0,183.0,28.0,74.0,1.0,524.0,0.408,0.423,0.818
97,203526,Raul,Neto,1610612739,CLE,188,2022-23,48.0,157.0,79.0,46.0,4.0,505.0,0.518,0.286,0.912
98,1629669,Jaylen,Nowell,1610612750,MIN,193,2022-23,65.0,704.0,132.0,170.0,5.0,1252.0,0.448,0.289,0.778


In [435]:
#players_stats_df_22 = agregar_estadisticas_temporada(all_players.drop(columns=['SEASON_ID']), '2021-22')
#players_stats_df_22.to_csv('jugadores_estadisticas_22.csv', index=False)

players_stats_df_22 = pd.read_csv('jugadores_estadisticas_22.csv')

In [436]:
players_stats_df_22_clean = players_stats_df_22.drop_duplicates(subset=['PERSON_ID'])
players_stats_df_22_clean

,PERSON_ID,NOMBRE,APELLIDO,TEAM_ID,TEAM_ABBREVIATION,ALTURA_CM,SEASON_ID,GP,PTS,AST,REB,BLK,MIN,FG_PCT,FG3_PCT,FT_PCT
0,200746,LaMarcus,Aldridge,1610612751,BKN,211,2021-22,47,607,42,258,47,1050,0.550,0.304,0.873
1,2546,Carmelo,Anthony,1610612747,LAL,201,2021-22,69,919,68,288,52,1793,0.441,0.375,0.830
2,1627853,Ryan,Arcidiacono,1610612752,NYK,190,2021-22,10,16,4,8,0,76,0.500,0.444,0.000
3,201571,D.J.,Augustin,1610612747,LAL,180,2021-22,34,182,74,41,0,510,0.404,0.406,0.868
5,203145,Kent,Bazemore,1610612747,LAL,193,2021-22,39,132,34,69,9,545,0.324,0.363,0.765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,1630177,Theo,Maledon,1610612766,CHA,193,2021-22,51,361,111,131,11,908,0.375,0.293,0.790
98,203585,Rodney,McGruder,1610612765,DET,193,2021-22,51,273,48,111,7,754,0.436,0.397,0.731
99,203526,Raul,Neto,1610612739,CLE,188,2021-22,70,526,217,134,3,1372,0.463,0.292,0.769
100,1629669,Jaylen,Nowell,1610612750,MIN,193,2021-22,62,527,130,123,10,975,0.475,0.394,0.783


In [437]:
all_players_stats = pd.concat([players_stats_df_22_clean, players_stats_df_23_clean], ignore_index=True)


In [438]:
all_players_stats.to_csv('jugadores_estadisticas.csv', index=False)

In [439]:
all_players_stats = pd.read_csv('jugadores_estadisticas.csv')
all_players_stats = all_players_stats[['PERSON_ID', 'SEASON_ID', 'GP', 'PTS', 'AST', 'REB','BLK']]

In [440]:
all_players_stats = all_players_stats.dropna(subset=['SEASON_ID'])

In [441]:
all_players_stats 


,PERSON_ID,SEASON_ID,GP,PTS,AST,REB,BLK
0,200746,2021-22,47.0,607.0,42.0,258.0,47.0
1,2546,2021-22,69.0,919.0,68.0,288.0,52.0
2,1627853,2021-22,10.0,16.0,4.0,8.0,0.0
3,201571,2021-22,34.0,182.0,74.0,41.0,0.0
4,203145,2021-22,39.0,132.0,34.0,69.0,9.0
...,...,...,...,...,...,...,...
193,1630177,2022-23,44.0,295.0,152.0,123.0,12.0
194,203585,2022-23,32.0,183.0,28.0,74.0,1.0
195,203526,2022-23,48.0,157.0,79.0,46.0,4.0
196,1629669,2022-23,65.0,704.0,132.0,170.0,5.0


In [442]:
import psycopg2
from datetime import datetime

# Conexión a la base de datos
conn = psycopg2.connect(
    host="localhost",
    dbname="nba_db",
    user="postgres",
    password="12345"
)
cursor = conn.cursor()

In [ ]:
for _, row in all_players_stats.iterrows():
    player_id = row['PERSON_ID']
    season_id = row['SEASON_ID']
    gp = int(row['GP'] if not pd.isna(row['GP']) else 0)
    pts = int(row['PTS'] if not pd.isna(row['PTS']) else 0)
    ast = int(row['AST'] if not pd.isna(row['AST']) else 0)
    reb = int(row['REB'] if not pd.isna(row['REB']) else 0)
    blk = int(row['BLK'] if not pd.isna(row['BLK']) else 0)

    cursor.execute("""
        INSERT INTO EstadisticasJugador (id_jugador, NombreTemporada, TotalJuegos, Puntos, Asistencias, Rebotes, Bloqueos)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (id_jugador, NombreTemporada) DO NOTHING
    """, (player_id, season_id, gp, pts, ast, reb, blk))

conn.commit()

## Llenando Tabla EstadisticasEquipo

In [445]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    dbname="nba_db", # <- Aqui va el nombre de tu base de datos
    user="postgres", # <- Aqui va tu usuario de pgadmin
    password="12345" # <- Aqui va la contraseña
)
cursor = conn.cursor()

In [ ]:
from nba_api.stats.endpoints import teamyearbyyearstats 

temporadas = ['2021-22', '2022-23']

for temporada in temporadas:
    print(f"\nProcesando temporada {temporada}")
    for id in abbrev_a_id.values():
        try:
            stats = teamyearbyyearstats.TeamYearByYearStats(team_id=str(id)).get_data_frames()[0]
            stats_temp = stats[stats['YEAR'] == temporada]

            if stats_temp.empty:
                print(f"No hay datos para el equipo {id} en la temporada {temporada}")
                continue

            # Solo una fila, extraemos con .iloc[0]
            row = stats_temp.iloc[0]

            NombreTemporada = row['YEAR']
            Equipo = int(row['TEAM_ID'])
            TotalJuegos = int(row['GP'])
            Ganados = int(row['WINS'])
            Perdidos = int(row['LOSSES'])
            PlayOffGanados = int(row['PO_WINS'])
            PlayoffPerdidos = int(row['PO_LOSSES'])
            Finales = row['NBA_FINALS_APPEARANCE'] in ['FINALS APPEARANCE', 'LEAGUE CHAMPION']

            cursor.execute("""
                INSERT INTO estadisticasEquipo (
                    NombreTemporada, Equipo, TotalJuegos, Ganados, Perdidos,
                    PlayOffGanados, PlayoffPerdidos, Finales)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                ON CONFLICT (Equipo, NombreTemporada) DO NOTHING
            """, (
                NombreTemporada, Equipo, TotalJuegos, Ganados, Perdidos,
                PlayOffGanados, PlayoffPerdidos, Finales
            ))

        except Exception as e:
            print(f"Error con equipo {id} en {temporada}: {e}")
            conn.rollback()
        else:
            conn.commit()
            print(f"{id} - {temporada} insertado")

        time.sleep(1.2)

cursor.close()
conn.close()


Procesando temporada 2021-22
1610612737 - 2021-22 insertado
1610612738 - 2021-22 insertado
1610612739 - 2021-22 insertado
1610612740 - 2021-22 insertado
1610612741 - 2021-22 insertado
1610612742 - 2021-22 insertado
1610612743 - 2021-22 insertado
1610612744 - 2021-22 insertado
1610612745 - 2021-22 insertado
1610612746 - 2021-22 insertado
1610612747 - 2021-22 insertado
1610612748 - 2021-22 insertado
1610612749 - 2021-22 insertado
1610612750 - 2021-22 insertado
1610612751 - 2021-22 insertado
1610612752 - 2021-22 insertado
1610612753 - 2021-22 insertado
1610612754 - 2021-22 insertado
1610612755 - 2021-22 insertado
1610612756 - 2021-22 insertado
1610612757 - 2021-22 insertado
1610612758 - 2021-22 insertado
1610612759 - 2021-22 insertado
1610612760 - 2021-22 insertado
1610612761 - 2021-22 insertado
1610612762 - 2021-22 insertado
1610612763 - 2021-22 insertado
1610612764 - 2021-22 insertado
1610612765 - 2021-22 insertado
1610612766 - 2021-22 insertado

Procesando temporada 2022-23
1610612737